## 对比生成器和协成

In [1]:
def generator_func():
    yield

gen = generator_func()
print(gen)
print(sorted(set(dir(gen)) - set(dir(object))))

<generator object generator_func at 0x106baa510>
['__del__', '__iter__', '__name__', '__next__', '__qualname__', 'close', 'gi_code', 'gi_frame', 'gi_running', 'gi_yieldfrom', 'send', 'throw']


In [2]:
async def continue_func():
    pass

coro = continue_func()
print(coro)
print(sorted(set(dir(coro)) - set(dir(object))))

<coroutine object continue_func at 0x106ba6ac0>
['__await__', '__del__', '__name__', '__qualname__', 'close', 'cr_await', 'cr_code', 'cr_frame', 'cr_origin', 'cr_running', 'send', 'throw']


## yield表达式获取值

In [3]:
def show_yield_value():
    print("start")
    x = yield 100
    print(f"x is {x}")

In [5]:
g = show_yield_value()
next(g)

start


100

In [6]:
next(g)

x is None


StopIteration: 

In [7]:
g = show_yield_value()
g.send(None)

start


100

In [8]:
x = g.send("hello")

x is hello


StopIteration: 

## yield表达式优先级

In [9]:
def add_yield_value():
    x = yield + 1
    print(f"x is {x}")

In [10]:
g = add_yield_value()
next(g)

1

In [11]:
g.send(10)

x is 10


StopIteration: 

In [15]:
def add_yield_value():
    x = (yield) + 1
    print(f"x is {x}")

In [16]:
g = add_yield_value()
g.send(None)

In [17]:
g.send(2)

x is 3


StopIteration: 

## 极简echo

In [20]:
def gen_echo():
    while True:
        print((yield))

echo = gen_echo()
echo.send(None)

In [19]:
for action in ["吃饭", "睡觉", "跑步"]:
    echo.send(action)

吃饭
睡觉
跑步


In [21]:
echo.send(StopIteration("stop"))

stop


## 释放资源

In [22]:
def gen_echo_v2():
    while True:
        try:
            x = yield
        except GeneratorExit:
            print("exit, bye")
            return
        else:
            print(x)

echo = gen_echo_v2()
echo.send(None)

In [23]:
echo.send("akjdha")

akjdha


In [24]:
echo.close()

exit, bye


## 使用throw将异常抛给yield

In [25]:
def gen_echo_v3():
    while True:
        try:
            x = yield
        except GeneratorExit:
            print("exit, bye")
            return
        except KeyboardInterrupt:
            print("按下了ctrl-c")
        else:
            print(x)

echo = gen_echo_v3()
echo.send(None)

In [26]:
echo.throw(KeyboardInterrupt)

按下了ctrl-c


## 计算移动平均值

In [27]:
def coro_average():
    """计算移动平均值"""
    count = 0
    total = 0
    avg = 0
    while True:
        try:
            # 每次运行到yield关键字暂停
            print("before")
            val = yield avg
            print(f"after val {val}")
        except GeneratorExit:
            return total, count, avg
        else:
            total += val
            count += 1
            avg = total / count

In [28]:
coro_g = coro_average()
coro_g.send(None)

before


0

In [29]:
coro_g.send(1)

after val 1
before


1.0

In [30]:
coro_g.send(3)

after val 3
before


2.0